In [1]:
import os
import hashlib
from PIL import Image, UnidentifiedImageError, ImageFilter

def read_images_categories(image_dir, num_imgs_per_categorie):
    directories = ["drawings", "engraving", "iconography", "painting", "sculpture"]
    categories = []
    images = []
    filenames = []
    
    seen_hashes = set() 
    print(f"Iniciando carga y preprocesamiento desde: {image_dir}")

    for directory in directories:
        path = os.path.join(image_dir, directory)
        
        if not os.path.isdir(path):
            print(f"Advertencia: La carpeta {path} no existe. Saltando.")
            continue

        print(f"Procesando clase: {directory}...")
        count = 0
        
        files = os.listdir(path)
        
        for img_name in files:
            if count >= num_imgs_per_categorie:
                break
            
            try:
                img_path = os.path.join(path, img_name)
                
                img = Image.open(img_path)
                
                img = img.convert('RGB')
                
                img_hash = hashlib.md5(img.tobytes()).hexdigest()
                if img_hash in seen_hashes:
                    print(f" -> Duplicado detectado e ignorado: {img_name}")
                    continue 
                seen_hashes.add(img_hash)

                img = img.filter(ImageFilter.SMOOTH) 
                
                images.append(img)
                categories.append(directory)
                filenames.append(img_name)
                count += 1

            except (UnidentifiedImageError, OSError):
                print(f" -> Error leyendo o procesando: {img_name}")
                continue

    print(f"Proceso finalizado. Total imágenes limpias: {len(images)} | Total etiquetas: {len(categories)}")
    
    return images, categories, filenames

In [2]:
import json

ruta_dataset = "../data/dataset/training_set" 

cantidad_por_clase = 400

mis_imagenes, mis_etiquetas, mis_rutas = read_images_categories(ruta_dataset, cantidad_por_clase)

if len(mis_imagenes) > 0:
    print("\n--- Resultado ---")
    print(f"Total imágenes cargadas: {len(mis_imagenes)}")
    print(f"Dimensiones de la primera imagen: {mis_imagenes[0].size}")
    print(f"Etiqueta de la primera imagen: {mis_etiquetas[0]}")
    print(f"Etiqueta de la última imagen: {mis_etiquetas[-1]}")
else:
    print("No se cargaron imágenes. Revisa que la 'ruta_dataset' sea correcta.")

Iniciando carga y preprocesamiento desde: ../data/dataset/training_set
Procesando clase: drawings...
Procesando clase: engraving...
 -> Error leyendo o procesando: 249.jpg
 -> Error leyendo o procesando: 304.jpg
 -> Duplicado detectado e ignorado: 354.jpg
 -> Error leyendo o procesando: 356.jpg
 -> Duplicado detectado e ignorado: 357.jpg
Procesando clase: iconography...
 -> Error leyendo o procesando: 331.jpg
Procesando clase: painting...
 -> Error leyendo o procesando: 0000.jpg
 -> Duplicado detectado e ignorado: 0013.jpg
 -> Error leyendo o procesando: 0025.jpg
 -> Duplicado detectado e ignorado: 0038.jpg
 -> Duplicado detectado e ignorado: 0040.jpg
 -> Duplicado detectado e ignorado: 0043.jpg
 -> Duplicado detectado e ignorado: 0044.jpg
 -> Duplicado detectado e ignorado: 0045.jpg
 -> Duplicado detectado e ignorado: 0046.jpg
 -> Duplicado detectado e ignorado: 0047.jpg
 -> Duplicado detectado e ignorado: 0048.jpg
 -> Duplicado detectado e ignorado: 0049.jpg
 -> Error leyendo o proce

c:\Users\OMEN\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


 -> Duplicado detectado e ignorado: 362.jpg
 -> Duplicado detectado e ignorado: 363.jpg
 -> Duplicado detectado e ignorado: 364.jpg
 -> Duplicado detectado e ignorado: 366.jpeg
 -> Error leyendo o procesando: 374.jpg
 -> Duplicado detectado e ignorado: 378.jpeg
 -> Duplicado detectado e ignorado: 380.jpg
 -> Duplicado detectado e ignorado: 381.jpg
 -> Duplicado detectado e ignorado: 383.jpg
 -> Duplicado detectado e ignorado: 384.jpg
 -> Duplicado detectado e ignorado: 388.jpg
 -> Duplicado detectado e ignorado: 392 18.59.45.jpg
 -> Duplicado detectado e ignorado: 397.jpg
 -> Duplicado detectado e ignorado: 402.jpg
 -> Duplicado detectado e ignorado: 404.jpg
 -> Duplicado detectado e ignorado: 412.jpg
 -> Duplicado detectado e ignorado: 415.png
 -> Duplicado detectado e ignorado: 418.jpg
 -> Duplicado detectado e ignorado: 421.jpg
 -> Duplicado detectado e ignorado: 61.jpeg
 -> Duplicado detectado e ignorado: 9.jpg
Proceso finalizado. Total imágenes limpias: 2000 | Total etiquetas: 200

In [3]:
import torch
import numpy as np
import faiss
import gc  # Garbage Collector para liberar memoria entre modelos, al inicio era lento o se quedaba sin memoria
from torchvision import transforms
from feature_extractor import MyVGG16, MyResNet50, MyInceptionV3, MyColorHistogram, MyLBP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Lista de modelos a ejecutar: (Nombre_Archivo, Clase)
modelos_ = [
    ("VGG16", MyVGG16),
    ("ResNet50", MyResNet50),
    ("InceptionV3", MyInceptionV3), 
    ("ColorHistogram", MyColorHistogram),
    ("LBP", MyLBP)
]

# Preprocesamiento 
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

print("Preprocesando imágenes (se hace una sola vez)...")
input_batch = torch.stack([preprocess(img) for img in mis_imagenes])
print(f"Tensor de entrada completo: {input_batch.shape}")

num_images = input_batch.shape[0]
batch_size = 32 # No saturar la memoria 

for nombre_modelo, ClaseModelo in modelos_:
    print(f"\n================================================")
    print(f" INICIANDO PROCESO PARA: {nombre_modelo}")
    print(f"================================================")

    extractor = ClaseModelo(device=device)
    
    all_features_list = []

    # Extracción por lotes
    print(f"Extrayendo características con {nombre_modelo}...")
    for i in range(0, num_images, batch_size):
        batch = input_batch[i : i + batch_size]
        
        batch = batch.to(device)
        
        features = extractor.extract_features(batch)
        all_features_list.append(features)
        
        if i % (batch_size * 5) == 0:
            print(f"  -> Procesado lote {i} a {min(i + batch_size, num_images)}")

    # Concatenar resultados
    final_features = np.vstack(all_features_list)
    print(f"Extracción completada. Forma final: {final_features.shape}")

    # 4. Crear y Guardar índice FAISS
    features_db = final_features.astype('float32')
    d = features_db.shape[1] # Dimensión de los vectores 
    index = faiss.IndexFlatL2(d)
    print(f"Entrenando índice FAISS (dim={d})...")
    index.add(features_db)
    
    output_path = f"../Data/feature/{nombre_modelo}.index"
    faiss.write_index(index, output_path)
    print(f"Índice guardado exitosamente en: {output_path}")

    # Limpieza de memoria
    del extractor
    del final_features
    del features_db
    del index
    torch.cuda.empty_cache() # Libera caché 
    gc.collect() # Garbage collector de Python, teniamos problemas de memoria
    print(f"Memoria liberada. Listo para el siguiente modelo.")

print("\n------------------------------------------------")
print("Proceso completado para todos los modelos.")

Usando dispositivo: cpu
Preprocesando imágenes (se hace una sola vez)...
Tensor de entrada completo: torch.Size([2000, 3, 224, 224])

 INICIANDO PROCESO PARA: VGG16
Extrayendo características con VGG16...
  -> Procesado lote 0 a 32
  -> Procesado lote 160 a 192
  -> Procesado lote 320 a 352
  -> Procesado lote 480 a 512
  -> Procesado lote 640 a 672
  -> Procesado lote 800 a 832
  -> Procesado lote 960 a 992
  -> Procesado lote 1120 a 1152
  -> Procesado lote 1280 a 1312
  -> Procesado lote 1440 a 1472
  -> Procesado lote 1600 a 1632
  -> Procesado lote 1760 a 1792
  -> Procesado lote 1920 a 1952
Extracción completada. Forma final: (2000, 25088)
Entrenando índice FAISS (dim=25088)...
Índice guardado exitosamente en: ../Data/feature/VGG16.index
Memoria liberada. Listo para el siguiente modelo.

 INICIANDO PROCESO PARA: ResNet50
Extrayendo características con ResNet50...
  -> Procesado lote 0 a 32
  -> Procesado lote 160 a 192
  -> Procesado lote 320 a 352
  -> Procesado lote 480 a 512
 

100%|██████████| 104M/104M [00:01<00:00, 66.0MB/s] 


Extrayendo características con InceptionV3...
  -> Procesado lote 0 a 32
  -> Procesado lote 160 a 192
  -> Procesado lote 320 a 352
  -> Procesado lote 480 a 512
  -> Procesado lote 640 a 672
  -> Procesado lote 800 a 832
  -> Procesado lote 960 a 992
  -> Procesado lote 1120 a 1152
  -> Procesado lote 1280 a 1312
  -> Procesado lote 1440 a 1472
  -> Procesado lote 1600 a 1632
  -> Procesado lote 1760 a 1792
  -> Procesado lote 1920 a 1952
Extracción completada. Forma final: (2000, 2048)
Entrenando índice FAISS (dim=2048)...
Índice guardado exitosamente en: ../Data/feature/InceptionV3.index
Memoria liberada. Listo para el siguiente modelo.

 INICIANDO PROCESO PARA: ColorHistogram
Extrayendo características con ColorHistogram...
  -> Procesado lote 0 a 32
  -> Procesado lote 160 a 192
  -> Procesado lote 320 a 352
  -> Procesado lote 480 a 512
  -> Procesado lote 640 a 672
  -> Procesado lote 800 a 832
  -> Procesado lote 960 a 992
  -> Procesado lote 1120 a 1152
  -> Procesado lote 12

In [4]:
metadata = []

for filename, category in zip(mis_rutas, mis_etiquetas):
    metadata.append({
        "filename": filename,
        "category": category,
        "path": f"Data/dataset/training_set/{category}/{filename}" 
    })

with open('../Data/image_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=4)

print("Metadatos guardados como 'image_metadata.json'")

Metadatos guardados como 'image_metadata.json'
